In [1]:
import sys
import os

project_root = os.path.abspath('../..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from catan.core import Game
from catan.core.models.player import RandomPlayer, Color

In [11]:
players = [
    RandomPlayer(
        color=Color.BLUE,
        is_bot=True
    ),
    RandomPlayer(
        color=Color.RED,
        is_bot=True
    )
]

game = Game(players=players, seed=4)

game.play()

game.state.playable_actions

[Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.END_TURN: 'END_TURN'>, value=None)]

In [9]:
import catan.bots.mcts as mcts

players = [
    RandomPlayer(
        color=Color.BLUE,
        is_bot=True
    ),
    RandomPlayer(
        color=Color.RED,
        is_bot=True
    )
]
current_game_state: mcts.MCTSGameState = mcts.MCTSCatanGameState(
    players=players,
    seed=10
)
mcts_agent = mcts.MCTS(exploration_constant=1.414)

num_mcts_simulations = 10


for _ in range(20):
    player_to_move = current_game_state.get_current_player()
    best_action = mcts_agent.simulate(current_game_state, num_mcts_simulations)
    current_game_state = current_game_state. take_action(best_action)
    
    print(best_action)

Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=7)
Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_ROAD: 'BUILD_ROAD'>, value=(6, 7))
Action(color=<Color.RED: 'RED'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=11)
Action(color=<Color.RED: 'RED'>, action_type=<ActionType.BUILD_ROAD: 'BUILD_ROAD'>, value=(11, 32))
Action(color=<Color.RED: 'RED'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=15)
Action(color=<Color.RED: 'RED'>, action_type=<ActionType.BUILD_ROAD: 'BUILD_ROAD'>, value=(4, 15))
Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>, value=50)
Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.BUILD_ROAD: 'BUILD_ROAD'>, value=(49, 50))
Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.ROLL: 'ROLL'>, value=None)
Action(color=<Color.BLUE: 'BLUE'>, action_type=<ActionType.END_TURN: 'END_TURN'>, valu